In [1]:
import requests
import json
import urllib
import hashlib

/opt/conda/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def get_access_token(non_sso_user, non_sso_password):
    """To get access token"""
    
    password = hashlib.md5(non_sso_password.encode('utf-8')).hexdigest()
    username = non_sso_user
    request_url = "https://keycloak-qa.lti-mosaic.com/auth/realms/mosaic/protocol/openid-connect/token"
    params = [
    ("grant_type", "password"), ("client_id", "keycloak-gatekeeper"),
        ("client_secret", "62419864-bd9b-4d34-a235-f63fc8bedb95"),("username", f"{username}"),
        ("password", f"{password}")]
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    response = requests.post(request_url, data=params, headers=headers, verify=False)
    print("Access Response: ",response.status_code)
    return response.json()["access_token"]

In [3]:
def get_current_serving_image(env, docker_image_url, project_id, model_id, version_id):
    """"send put request to url with image."""
    """
    env : can be QA /GA /Cloud
    docker_image_url = keras or pytorch
    """
    
    access_token = get_access_token(non_sso_user, non_sso_password)
    
    payload = {"docker_image_url":docker_image_url}
    req_header = {"X-Project-Id": project_id, "Content-Type": "application/json", "Accept":"application/json","Authorization":"Bearer %s" %access_token} 
    
    api = "/registry/api/v1/ml-model/{model_id}/version/{version_id}"
    
    
    env_url = env.lower() + api
    url = env_url.format(model_id=model_id,version_id=version_id)
    print(url)
    req_response = requests.get(url, data=json.dumps(payload), headers=req_header, verify = False)
    
    return req_response

In [4]:
def update_serving_image(env, docker_image_url, project_id, model_id, version_id):
    """"send put request to url with image."""
    """
    env : can be QA /GA /Cloud
    docker_image_url = keras or pytorch
    """
    
    access_token = get_access_token(non_sso_user, non_sso_password)
    
    payload = {"docker_image_url":docker_image_url}
    req_header = {"X-Project-Id": project_id, "Content-Type": "application/json", 
                  "Accept":"application/json","Authorization":"Bearer %s" %access_token} 
    
    api = "/registry/api/v1/ml-model/{model_id}/version/{version_id}"
    
    
    env_url = env.lower() + api
    url = env_url.format(model_id=model_id,version_id=version_id)
    print(url)

    req_response = requests.put(url, data=json.dumps(payload), headers=req_header, verify = False)
    
    return req_response

In [5]:
def update_old_models(env, docker_image_url, project_id, model_id, version_id):
    """"send put request to url for old models to update image and removes the init_script"""
    
    """
    env : can be QA /GA /Cloud
    docker_image_url = keras or pytorch
    """
    
    access_token = get_access_token(non_sso_user, non_sso_password)
    
    payload = {"docker_image_url":docker_image_url, "init_script": ""}
    
    req_header = {"X-Project-Id": project_id, "Content-Type": "application/json", 
                  "Accept":"application/json","Authorization":"Bearer %s" %access_token} 
    
    api = "/registry/api/v1/ml-model/{model_id}/version/{version_id}"
    
    
    env_url = env.lower() + api
    url = env_url.format(model_id=model_id,version_id=version_id)
    print(url)

    req_response = requests.put(url, data=json.dumps(payload), headers=req_header, verify = False)
    
    return req_response

In [7]:
def tasks_to_perform(url, image, project_id, model_id, version_id, old_model=False): 
    """
    If we have old model which was not trained on localhost, we can use old_model = True. 
    This will delete the init script table. As we are fetching the current model version by current serving,
    the old init script will be printed which can be saved for reference by user.
    """
    
    if not old_model:
        
        """Fetching current to have backup of script displayed"""
        current_serving_resp = get_current_serving_image(url, image, project_id, model_id, version_id)
        print(current_serving_resp)
        current_serving_resp_json =  json.loads(current_serving_resp.text)
        for key in current_serving_resp_json:
            if key in ["name", "id", "ml_model_id", "current_version","docker_image_url","init_script"]:
                print(key," : ",current_serving_resp_json[key])
                print()

        resp = update_serving_image(url, image, project_id, model_id, version_id)
        print(resp)


    elif old_model:

        """Fetching current to have backup of script displayed"""
        current_serving_resp = get_current_serving_image(url, image, project_id, model_id, version_id)
        print(current_serving_resp)
        current_serving_resp_json =  json.loads(current_serving_resp.text)
        for key in current_serving_resp_json:
            if key in ["name", "id", "ml_model_id", "current_version","docker_image_url","init_script"]:
                print(key," : ",current_serving_resp_json[key])
                print()

        resp = update_old_models(url, image, project_id, model_id, version_id)
        print(resp)
        
    return resp, current_serving_resp_json

In [8]:
qa_project_id="9968fe47-6cd0-47b1-aab8-bdb4b091c116"


qa="https://mosaic-qa.lti-mosaic.com"
ga="https://livmosaic.lntinfotech.com"
cloud="https://mosaiccloud.lntinfotech.com"

non_sso_user="sanchit"
non_sso_password="123456"
keras="registry.lti-aiq.in:443/mosaic-ai-logistics/mosaic-ai-serving:1.0.0-10112021_uWSGI8_T1V1-EEF-Static"

In [21]:
list_models = [{"model_id" : "91c56fc3-c3fa-4356-8cad-2f451eced177",
                "version_id" : "0b95fa88-47b1-4b97-aee1-a1973ab27f84"}]

In [23]:
for models in list_models:
    
    print(models)
    print()
    
    update_response, old_serving_json = tasks_to_perform(qa, keras, 
                                                     qa_project_id, models["model_id"], models["version_id"], old_model=False)

{'model_id': '91c56fc3-c3fa-4356-8cad-2f451eced177', 'version_id': '0b95fa88-47b1-4b97-aee1-a1973ab27f84'}



/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'keycloak-qa.lti-mosaic.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Access Response:  200
https://mosaic-qa.lti-mosaic.com/registry/api/v1/ml-model/91c56fc3-c3fa-4356-8cad-2f451eced177/version/0b95fa88-47b1-4b97-aee1-a1973ab27f84


/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mosaic-qa.lti-mosaic.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<Response [200]>
docker_image_url  :  registry.lti-aiq.in:443/pranjalishirke/mosaic-ai-serving:uWSGI8_T1V1-EEF

id  :  0b95fa88-47b1-4b97-aee1-a1973ab27f84

init_script  :  "pip install --user -i localhost absl-py==0.12.0\n pip install --user -i localhost alabaster==0.7.12\n pip install --user -i localhost alembic==1.6.5\n pip install --user -i localhost alibi-detect==0.6.2\n pip install --user -i localhost antlr4-python3-runtime==4.7.1\n pip install --user -i localhost anyio==3.1.0\n pip install --user -i localhost anyjson==0.3.3\n pip install --user -i localhost APScheduler==3.6.0\n pip install --user -i localhost argon2-cffi==20.1.0\n pip install --user -i localhost asn1crypto==0.24.0\n pip install --user -i localhost astor==0.8.1\n pip install --user -i localhost astroid==2.3.3\n pip install --user -i localhost astunparse==1.6.3\n pip install --user -i localhost async-generator==1.10\n pip install --user -i localhost atomicwrites==1.3.0\n pip install --user -i localhost attrs==21.2

/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'keycloak-qa.lti-mosaic.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Access Response:  200
https://mosaic-qa.lti-mosaic.com/registry/api/v1/ml-model/91c56fc3-c3fa-4356-8cad-2f451eced177/version/0b95fa88-47b1-4b97-aee1-a1973ab27f84


/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mosaic-qa.lti-mosaic.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<Response [200]>
